<a href="https://colab.research.google.com/github/Josogrephy/Google-Map-Summarizer/blob/main/Google%20Map%20Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 233 kB in 2s (137 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
24 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building

In [3]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()


'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/120/chromedriver'

In [3]:
! pip3 install --upgrade --user google-cloud-aiplatform

import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:

import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

if "google.colab" in sys.modules:
    # Define project information
    PROJECT_ID = ""  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}

    # Initialize Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# specify the url of the business page on Google
google_map_url = 'https://www.google.com/maps/place/Orchard+Rd/@1.3048259,103.8296235,17z/data=!3m1!4b1!4m6!3m5!1s0x31da199148e6febb:0x8e99f73f37fef4fa!8m2!3d1.3048205!4d103.8321984!16zL20vMDN3MGw1?entry=ttu' # @param {type:"string"}

# create an instance of the Chrome driver
driver = webdriver.Chrome(options=chrome_options)
# navigate to the specified url
driver.get(google_map_url)

# Wait for the reviews to load
wait = WebDriverWait(driver, 20) # increased the waiting time
# document.getElementById("searchboxinput").value Title of the place
review_elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'wiI7pd')))
review_head = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'DUwDvf.lfPIob')))
review_img = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/button/img')))


# extract the text of each review
reviews = [element.text for element in review_elements]

# extract the text of each header
reviewsHead = [element.text for element in review_head]
reviewsHead = str(reviewsHead)[2:-2]

# extract the src of img
reviewImg = [element.get_attribute("src") for element in review_img]

# print the reviews
print(reviews)
print(reviewsHead)
print(reviewImg)

# close the browser
driver.quit()

['The most beautiful street during Christmas and New Year.\nMany shopping centres, food stalls, restaurants, shops, services, drinking places, and many other shops. …', 'Good street for shopping in Singapore. Well decorated for Christmas now.', 'It looks like Main Street. It is a good place for shopping as there are many shops selling luxury brand goods. My pictures have F1 event because of September.']
Orchard Rd
['https://lh5.googleusercontent.com/p/AF1QipN9-kPa4j_zUplT__0SBZrznNYxnePKAl5_-Y5Y=w408-h306-k-no']


In [6]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

model = GenerativeModel("gemini-pro")
print(reviews)
responses = model.generate_content("can you summarizes this in 1 paragraph from a first person view " + str(reviews), stream=True)

summaryFinal = ""
for response in responses:
    summaryFinal = summaryFinal + response.text

print(summaryFinal)

['The most beautiful street during Christmas and New Year.\nMany shopping centres, food stalls, restaurants, shops, services, drinking places, and many other shops. …', 'Good street for shopping in Singapore. Well decorated for Christmas now.', 'It looks like Main Street. It is a good place for shopping as there are many shops selling luxury brand goods. My pictures have F1 event because of September.']
Strolling down the bustling streets of Orchard Road during the festive season is a magical experience. The street is adorned with dazzling Christmas decorations, turning it into a winter wonderland. Numerous shopping malls, food stalls, restaurants, and shops line the road, offering a wide variety of options for shopping, dining, and entertainment. Whether you're looking for luxury brands or souvenirs, or simply want to soak in the праздничная атмосфера, Orchard Road is the place to be.


In [7]:
from string import Template
htmlTemplate = Template('<!DOCTYPE html> <html> <head> <meta charset="utf-8" /> </head> <body> <script type="application/json" data-props> { "post": { "metadata": { "creator": { "onePresenceId": "jsian" }, "title": "$where"},"view": {"viewType": "overlay","hero": {"blockType": "single-page","data": {"foreground": [{"presentation": {"position": {"x": 17, "y": 8 }},"blockType": "text","data": {"content": "$where"}}],"background": {"blockType": "image","data": {"src": "$img?auto=format&fit=crop&w=500&h=800&q=80"}}}},"addons": [{"blockType": "outlink","data": {"href": "$url","title": "$where | What other people say","text": "$summary","thumbnail": {"src": "$img?auto=format&fit=crop&w=200&h=200","alt": "madison square park photo"}}}]}}}</script><script async type="module" src="https://www.gstatic.com/modern-formats-eng/ucp/static/231215.0/csr/en-us/csr.mjs"></script></body></html>')
htmlFinal = htmlTemplate.safe_substitute(where=reviewsHead, summary=summaryFinal, img=reviewImg[0], url=google_map_url)
with open("index.html", "w") as file:
    file.write(htmlFinal)